In [1]:
import sys
sys.path.append('./src')
import KGBN
 

In [ ]:
network_string = """
EGF = EGF
EGFR = EGF
DNA_DAMAGE  = DNA_DAMAGE
KRAS   = EGFR
RAF1   = KRAS
MAP2K1 = RAF1
MAPK1  = MAP2K1
PIK3CA = EGFR | MAPK1
PTEN   = !PIK3CA
AKT1   = PIK3CA & !PTEN
MDM2   = AKT1
TP53   = DNA_DAMAGE & !MDM2
CASP3  = TP53 & !AKT1
CDK2   = (MAPK1 | AKT1) & !TP53
"""
network = KGBN.load_network_from_string(network_string)

In [102]:
# visulize a gene regulatory network

# the input is as follows:
logic_rules = {
    "TNF": "TNF",
    "EGF": "EGF",
    "EGFR": "EGF",
    "TNFRs": "! (TNF | NFkB)",
    "complexI": "TNFRs",
    "SOS": "EGFR | (! ERK)",
    "RAS": "SOS"
}

file = 'input_files/Eduati2020.txt'
file = 'input_files/fig3_AML_equations.txt'
file = 'input_files/dnmt3a_equations.txt'
file = 'input_files/Eduati2020.txt'
file = 'input_files/pancreatic_vundavilli_2020_fig3.txt'
file = 'input_files/Eduati2020.txt'
file = 'input_files/fig3_AML_equations.txt'
file = 'input_files/dnmt3a_equations.txt'
file = 'input_files/pancreatic_vundavilli_2020_fig3.txt'


# read the file as a logic rule dictionary
def read_logic_rules(file):
    logic_rules = {}
    with open(file, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue
            parts = line.strip().split('=')
            if len(parts) == 2:
                logic_rules[parts[0].strip()] = parts[1].strip()
    return logic_rules

logic_rules = read_logic_rules(file)



In [103]:

import igraph as ig
from pyvis.network import Network
import re



# Build the graph from rules
net = Network(notebook=True, directed=True, height='1000px', width='1200px') # Enable directed graph for arrows

def build_igraph(logic_rules):
    g = ig.Graph(directed=True)  # Ensure the graph is directed
    node_names = set(logic_rules.keys())

    # Also collect all appearing variables
    for rule in logic_rules.values():
        node_names.update(re.findall(r'\b[A-Za-z0-9_]+\b', rule))

    node_names = list(node_names)
    g.add_vertices(node_names)

    # Add directed edges
    for target, rule in logic_rules.items():
        inputs = set(re.findall(r'\b[A-Za-z0-9_]+\b', rule))
        for input_node in inputs:
            #if input_node != target:
            g.add_edge(input_node, target, label=rule)

    return g

g = build_igraph(logic_rules)

# Add nodes to the PyVis network
for v in g.vs:
    upstream_count = len([pred for pred in g.predecessors(v.index) if pred != v.index])  # Exclude self-predecessors
    downstream_count = len([succ for succ in g.successors(v.index) if succ != v.index])  # Exclude self-successors
    
    # Set color based on upstream and downstream node counts
    if upstream_count == 0:
        color = "grey"  # Source node
    elif downstream_count == 0:
        color = "orange"  # Sink node
    else:
        color = "lightblue"  # Intermediate node

    net.add_node(v.index, label=v["name"], title=v["name"], color=color, size=40, font_size=40, labelHighlightBold=True, physics=True)

# Add edges to the PyVis network
for e in g.es:
    src, tgt = e.tuple

    rule_label = e["label"]
    
    if rule_label.startswith("!(") and g.vs[src]["name"] in rule_label:
        # Handle the case where the rule starts with "!("
        net.add_edge(src, tgt, title=rule_label, color="grey", arrowStrikethrough=False)
    elif rule_label.startswith("! (") and g.vs[src]["name"] in rule_label:
        # Handle the case where the rule starts with "!"
        net.add_edge(src, tgt, title=rule_label, color="grey", arrowStrikethrough=False)
    elif "! (" in rule_label and g.vs[src]["name"] in rule_label:
        # Handle the case where the rule contains "! ("
        net.add_edge(src, tgt, title=rule_label, color="grey", arrowStrikethrough=False)
    elif f"!{g.vs[src]['name']}" in rule_label or f"! {g.vs[src]['name']}" in rule_label:
        # Add a separate edge for negation, considering direction
        net.add_edge(src, tgt, title=rule_label, color="blue", arrowStrikethrough=False)
    else:
        net.add_edge(src, tgt, title=rule_label, color="red", arrowStrikethrough=False)

# Enable physics (for dynamic layout & dragging)
net.toggle_physics(True)

# Show the graph
net.show("logic_graph.html")


logic_graph.html
